In [6]:
def set_default_if_empty(nested_dict, path, default):
    """
    Sets a default value in a nested dictionary based on a given path if the final key is not set or empty.
    
    :param nested_dict: The nested dictionary to modify.
    :param path: A list of keys representing the path to the target value.
    :param default: The default value to set if the target is not set or empty.
    """
    print("Set default if empty: ", path, default)
    if isinstance(nested_dict, list):
        return [set_default_if_empty(item, path[1:], default) for item in nested_dict]
    
    if isinstance(nested_dict, dict):
        # Navigate through the nested dictionary along the path, except for the last key
        current_level = nested_dict

        for key in path[:-1]:
            print("Key: ", key, "Isinstance: ", isinstance(current_level[key], list))
            if key in current_level:
                if current_level[key] is None or current_level[key] == "":
                    return current_level
                elif isinstance(current_level[key], list):
                    current_level[key] = [set_default_if_empty(item, path[1:], default) for item in current_level[key]]
                elif isinstance(current_level[key], dict):
                    current_level[key] = set_default_if_empty(current_level[key], path[1:], default)

        final_key = path[-1]
        if final_key in current_level and current_level[final_key] is not None:
            if (isinstance(current_level[final_key], str) or 
                isinstance(current_level[final_key], dict)) and isinstance(default, list):
                print("C: ", current_level[final_key])
                current_level[final_key] = [current_level[final_key]]

        return current_level

In [12]:
import json
from collections import defaultdict

def check_types(data, parent_key='', path_types=defaultdict(list)):
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                check_types(value, f"{parent_key}.{key}" if parent_key else key, path_types)
            else:
                path_types[f"{parent_key}.{key}" if parent_key else key].append(type(value).__name__)
    elif isinstance(data, list):
        for i, item in enumerate(data):
            check_types(item, f"{parent_key}[{i}]" if parent_key else str(i), path_types)
    else:
        path_types[parent_key].append(type(data).__name__)

    return path_types

def find_inconsistencies(path_types):
    inconsistencies = {}
    for path, types in path_types.items():
        if len(set(types)) > 1:  # More than one unique type for the path
            inconsistencies[path] = set(types)
    return inconsistencies

In [13]:
import json

data = json.load(open("/Users/jy006/Documents/Code/BioMedGPS/biomedgps/data/drugbank_5.1_2024-01-03.json"))

path_types = check_types(data)
inconsistencies = find_inconsistencies(path_types)
if inconsistencies:
    print("Found inconsistencies in the data types:")
    for path, types in inconsistencies.items():
        print(f"Path: {path}, Types: {types}")
else:
    print("No inconsistencies found in the data types.")

No inconsistencies found in the data types.
